<a href="https://colab.research.google.com/github/cryanwashere/cams_stuff/blob/main/Lesion_Segmentation_Complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext tensorboard

In [ ]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np 
import random
from IPython.display import clear_output
import tensorflow.keras.layers as tfl
import datetime

In [ ]:
rm -rf ./logs/

In [ ]:
image_path = 'drive/MyDrive/Data/ISIC2018_Task1-2_Training_Input'
mask_path = 'drive/MyDrive/Data/ISIC2018_Task1_Training_GroundTruth'
image_dir = os.listdir(image_path)
mask_dir = os.listdir(mask_path)
image_dir.sort()
mask_dir.sort()

In [ ]:
train_number = 1000

In [ ]:
images = []
count = 0
for i in image_dir:
  if i != 'ATTRIBUTION.txt':
    if count > train_number:
      break
    count+=1
    image = cv2.imread(os.path.join(image_path, i))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (128,128))
    images.append(image)
    clear_output(wait=True)
    print(count)

423


In [ ]:
masks = []
count = 0
for i in mask_dir:
  if i != 'ATTRIBUTION.txt':
    if count > train_number:
      break
    count+=1
    mask = cv2.imread(os.path.join(mask_path, i))
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    mask = cv2.resize(mask, (104,104))
    mask = np.expand_dims(mask, axis=-1)
    masks.append(mask)
    clear_output(wait=True)
    print(count)

In [ ]:
for i in range(len(images)):
  images[i] = images[i]/255
  masks[i] = masks[i]/255

images=np.array(images)
masks=np.array(masks)

In [ ]:
input_shape = ( 128, 128, 3)
input = tfl.Input(shape = (input_shape))
c0 = tfl.Conv2D(64, 3, activation = 'relu')(input)
c1 = tfl.Conv2D(64, 3, activation = 'relu')(c0)
m1 = tfl.MaxPool2D(pool_size = (2,2), strides = (2,2))(c1)
c2 = tfl.Conv2D(128, 3, activation = 'relu')(m1)
c3 = tfl.Conv2D(128, 3, activation = 'relu')(c2)
m2 = tfl.MaxPool2D(pool_size = (2,2), strides = (2,2))(c3)
c4 = tfl.Conv2D(256, 3, activation = 'relu')(m2)
c5 = tfl.Conv2D(256, 3, activation = 'relu')(c4)
m3 = tfl.MaxPool2D(pool_size = (2,2), strides = (2,2))(c5)
c6 = tfl.Conv2D(512, 3, activation = 'relu')(m3)
c7 = tfl.Conv2D(512, 3, activation = 'relu')(c6)
t1 = tfl.Conv2DTranspose(256, 2, strides = (3,3), activation = 'relu')(c7)
crop1 = tfl.Cropping2D(cropping = ((1,0), (1,0)))(c5)
concat1 = tfl.Concatenate(axis = -1)([t1, crop1])
c8 = tfl.Conv2D(256, 3, activation = 'relu')(concat1)
c9 = tfl.Conv2D(256, 3, activation = 'relu')(c8)
t2 = tfl.Conv2DTranspose(128, 2, strides = (2,2), activation = 'relu')(c9)
crop2 = tfl.Cropping2D(cropping = ((9,9), (9,9)))(c3)
concat2 = tfl.Concatenate(axis = -1)([t2, crop2])
c10 = tfl.Conv2D(128, 3, activation = 'relu')(concat2)
c11= tfl.Conv2D(128, 3, activation = 'relu')(c10)
t3 = tfl.Conv2DTranspose(64, 2, strides = (3,3), activation = 'relu')(c11)
crop3 = tfl.Cropping2D(cropping = ((8,8), (8,8)))(c1)
concat3 = tfl.Concatenate(axis = -1)([t3, crop3])
c12= tfl.Conv2D(64, 3, activation = 'relu')(concat3)
output = tfl.Conv2D(1, 3)(c12)

In [ ]:
model = tf.keras.Model(inputs = input, outputs = output)

In [ ]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

In [ ]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
history = model.fit(images, masks, epochs = 10, verbose=1, callbacks=[tensorboard_callback])

Epoch 1/10
32/32 [==============================] - 563s 17s/step - loss: 0.4572 - accuracy: 0.7526
Epoch 2/10
32/32 [==============================] - 556s 17s/step - loss: 0.3748 - accuracy: 0.8268
Epoch 3/10
32/32 [==============================] - 556s 17s/step - loss: 0.3520 - accuracy: 0.8411
Epoch 4/10
32/32 [==============================] - 557s 17s/step - loss: 0.3413 - accuracy: 0.8466
Epoch 5/10
32/32 [==============================] - 556s 17s/step - loss: 0.3441 - accuracy: 0.8491
Epoch 6/10
32/32 [==============================] - 557s 17s/step - loss: 0.3283 - accuracy: 0.8552
Epoch 7/10
32/32 [==============================] - 557s 17s/step - loss: 0.4113 - accuracy: 0.8248
Epoch 8/10
32/32 [==============================] - 557s 17s/step - loss: 0.3395 - accuracy: 0.8509
Epoch 9/10
32/32 [==============================] - 557s 17s/step - loss: 0.3337 - accuracy: 0.8550
Epoch 10/10
20/32 [=================>............] - ETA: 3:50 - loss: 0.3410 - accuracy: 0.8531

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
model.save('drive/MyDrive/Data/LesionSegmentationModel.h5')

In [ ]:
def clean_mask(image, precision):
  mask=model.predict(image[tf.newaxis,...])
  mask=np.expand_dims(tf.squeeze(mask),axis=-1)
  mask=cv2.resize(mask,(128,128))
  mask=np.expand_dims(mask,axis=-1)

  clean_mask=[]
  for i in mask:
    row=[]
    for j in i:
      if j>precision:
        row.append(1)
      else:
        row.append(0)
    clean_mask.append(row)
  clean_mask=np.expand_dims(clean_mask,axis=-1)
  return clean_mask

In [ ]:
plt.imshow(tf.squeeze(clean_mask(images[23], 0.1)))

In [ ]:
plt.imshow(images[23])

In [ ]:
[1,0.5,0.7]
[0.1,0.4,0.7]

In [ ]:
rand=random.randint(0,500)
image=images[rand]
mask=clean_mask(image, 0.3)
cleaned_image=[]

for i in range(128):
  row=[]
  for j in range(128):
    if mask[i][j]==1:
      row.append(image[i][j])
    else:
      row.append([0.1,0.4,0.7])
  cleaned_image.append(row)



print(np.shape(cleaned_image))
plt.subplot(1,2,1)
plt.imshow(cleaned_image)
plt.subplot(1,2,2)
plt.imshow(image)

In [ ]:
sample_path='drive/MyDrive/Data/leedsbutterfly/images/0010001.png'
sample_image=cv2.imread(sample_path)
sample_image=cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB)
sample_image=cv2.resize(sample_image,(128,128))
print(np.shape(sample_image))
plt.subplot(1,2,1)
plt.imshow(sample_image)
plt.subplot(1,2,2)
plt.imshow(tf.squeeze(model.predict(sample_image[tf.newaxis,...])))